In [ ]:
# initialise and test speech recognition
import speech_recognition as sr

test_filename = "sample.wav"

r = sr.Recognizer()
data = sr.AudioFile(test_filename)
with data as source:
    audio = r.record(source)

print(r.recognize_google(audio))

In [ ]:
import pyttsx3
tts_engine = pyttsx3.init()

tts_engine.say("checking tts")

tts_engine.runAndWait()

In [ ]:
# initialise and test Robot core

from RobotClass import Robot
import time

robot = Robot()

robot.forward(0.3)
time.sleep(0.25)
robot.backward(0.3)
time.sleep(0.25)
robot.stop()

In [ ]:
 # Find the correct microphone

device_label = "HD Pro Webcam C920" # for Logitech C920
#device_label = "USB 2.0 Camera: Audio" # for the Nulea cameras
#device_label = "USB Device 0x46d:0x825" # for the logitech c270 camera (non-functional)

try:
    del mic
except NameError:
    pass

for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print("\"{1}\" : {0}".format(index, name))
    if name.startswith(device_label):
        print("Found mic with target name")
        mic = sr.Microphone(device_index=index)
        break
else:
    print("Could not find specified microphone, using default")
    mic = sr.Microphone()       


In [34]:
# mk1 code block for voice control
import asyncio

async def say(text):
    tts_engine.say(text)
    await tts_engine.runAndWait()

async def perform_action(action):
    robot.set_motors(action[0],action[1],action[2],action[3])
    await asyncio.sleep(duration)
    robot.stop()

speed_mul = 0.5
turn_mul = 0.5
duration = 1
command_dict = {"forward":[speed_mul,speed_mul,speed_mul,speed_mul],
"backward":[-speed_mul,-speed_mul,-speed_mul,-speed_mul],
"left":[-turn_mul,turn_mul,-turn_mul,turn_mul],
"right":[turn_mul,-turn_mul,turn_mul,-turn_mul]}

similarities = {"backwood":"backward",
"back":"backward",
"light":"right"}

with mic as source:
    #r.adjust_for_ambient_noise(source)
    while True:
        try:
            audio = r.listen(source)

            print("stopped listening, sending audio...")
            recognised_text = r.recognize_google(audio)
            print("speech recognised as:",recognised_text)

            command = recognised_text.split(" ")[0]

            try:
                new_command = similarities[command]
                print("{} sounds pretty similar to {}, substituting.".format(command,new_command))
                command = new_command
            except KeyError:
                pass
            
            print("command:",command)

            try:
                action = command_dict[command]

                """
                tts_engine.say("performing command: {}".format(command))
                tts_engine.runAndWait()

                robot.set_motors(action[0],action[1],action[2],action[3])
                time.sleep(duration)
                robot.stop()
                """

                asyncio.gather(say("performing command: {}".format(command)),perform_action(action))

            except KeyError:
                tts_engine.say("I don't recognised the command: {}".format(command))
                tts_engine.runAndWait()
                


        except sr.UnknownValueError:
            pass

stopped listening, sending audio...
speech recognised as: forward
command: forward
stopped listening, sending audio...
stopped listening, sending audio...
stopped listening, sending audio...
speech recognised as: backwood
backwood sounds pretty similar to backward, substituting.
command: backward


KeyboardInterrupt: 